In [1]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('../../data/session-df-ak.csv')

In [2]:
cols1 = [ 'hour_of_day', 'session_position', 'premium', 'session_length','hist_user_behavior_is_shuffle', 'custom_skip'] # , 'custom_pause_before_play'
df1 = df[cols1]
for col in cols1:
    df1[col] = df1[col].astype(int)
df1.rename(columns={'hist_user_behavior_is_shuffle':'shuffle'},inplace =True)

c:\users\skadi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\skadi\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:4298: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [3]:
import graphviz
causal_graph = """digraph {
shuffle->{sesssion_position,custom_skip,session_length,premium};
session_length->{session_position, custom_skip, premium};
hour_of_day->session_length;
premium->custom_skip;
}
"""

In [20]:
import dowhy
model= dowhy.CausalModel(
        data = df1,
        graph=causal_graph.replace("\n", " "),
        treatment="shuffle",
        outcome='custom_skip')
model.view_model()
# from IPython.display import Image, display
# display(Image(filename="causal_model.png"))

In [21]:
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
    d                    
──────────(E[customₛₖᵢₚ])
d[shuffle]               
Estimand assumption 1, Unconfoundedness: If U→{shuffle} and U→custom_skip then P(custom_skip|shuffle,,U) = P(custom_skip|shuffle,)

### Estimand : 2
Estimand name: iv
No such variable(s) found!

### Estimand : 3
Estimand name: frontdoor
No such variable(s) found!



In [22]:
estimate = model.estimate_effect(identified_estimand,
                                 method_name="backdoor.linear_regression",target_units="ate")
# ATE = Average Treatment Effect
# ATT = Average Treatment Effect on Treated (i.e. those who were assigned a different room)
# ATC = Average Treatment Effect on Control (i.e. those who were not assigned a different room)
print(estimate)

linear_regression
{'control_value': 0, 'treatment_value': 1, 'test_significance': None, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['hour_of_day']}


c:\users\skadi\appdata\local\programs\python\python37\lib\site-packages\dowhy\causal_estimator.py:255: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_quantiles, duplicates="drop")


*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
    d                    
──────────(E[customₛₖᵢₚ])
d[shuffle]               
Estimand assumption 1, Unconfoundedness: If U→{shuffle} and U→custom_skip then P(custom_skip|shuffle,,U) = P(custom_skip|shuffle,)

## Realized estimand
b: custom_skip~shuffle+shuffle*hour_of_day
Target units: ate

## Estimate
Mean value: 0.03493388716692036
### Conditional Estimates
__categorical__hour_of_day
(-0.001, 10.0]    0.026105
(10.0, 13.0]      0.032809
(13.0, 16.0]      0.035748
(16.0, 20.0]      0.039202
(20.0, 23.0]      0.042690
dtype: float64
